# ISIC test notebook for isicarchive package

## Installation
To install the isicarchive package from github (more frequent
updates for now), please use
~~~~
pip install git+https://github.com/neuroelf/isicarchive
~~~~

Otherwise, you can download a relatively more stable version
from PyPi using
~~~~
pip install isicarchive
~~~~

In [1]:
# Make sure the latest isicarchive package is installed!
import sys
!{sys.executable} -m pip uninstall -y isicarchive
!{sys.executable} -m pip install git+https://github.com/neuroelf/isicarchive

Can't uninstall 'isicarchive'. No files were found to uninstall.
  Cloning https://github.com/neuroelf/isicarchive to c:\users\weberj3\appdata\local\temp\pip-req-build-lsyx_f3x
  Stored in directory: C:\Users\weberj3\AppData\Local\Temp\pip-ephem-wheel-cache-9oeabx7f\wheels\ba\cf\32\48caf3dba09770da83411064c00c38010306f48fd3bcdb784a
Successfully built isicarchive
  Found existing installation: isicarchive 0.2.3
    Can't uninstall 'isicarchive'. No files were found to uninstall.


  Running command git clone -q https://github.com/neuroelf/isicarchive 'C:\Users\weberj3\AppData\Local\Temp\pip-req-build-lsyx_f3x'


## Import and use the ```IsicApi``` object from the isicarchive module

### ***Please change the username and cache_folder!***

In [2]:
# Import isicapi
from isicarchive import IsicApi, func

# Set username and password (to None!)
username = 'weberj3@mskcc.org'
password = None
cache_folder = 'C:\\Users\\weberj3\\Documents\\git\\ISIC\\cache'
#cache_folder = '/Data/MSKCC/isicarchive/cache'

# Instantiate API, will ask for password using getpass.getpass(...)
api = IsicApi(username, password, cache_folder=cache_folder)

# load cached images
api.cache_images()

# pretty print object representation
api

Password for "weberj3@mskcc.org":········


isicarchive.api.IsicApi({
    'base_url': 'https://isic-archive.com/api/v1',
    'username': 'weberj3@mskcc.org',
    'cache_folder': 'C:\Users\weberj3\Documents\git\ISIC\cache',
    'cached_images': { ... dict with 68708 fields},
    'loaded_datasets': { ... dict with 29 fields},
    'loaded_studies': { ... dict with 18 fields},
    'obj_annotations': { ... dict with 0 fields},
    'obj_datasets': { ... dict with 0 fields},
    'obj_images': { ... dict with 0 fields},
    'obj_studies': { ... dict with 0 fields},})

## Example usage to retrieve a study and an image from the study

In [3]:
# Retrieve a study
study = api.study('EASY Study - 75 Image, full featureset')

# load annotations
study.load_annotations()

# print some information about the study
study

isicarchive.study.Study({
    'id': '58d9189ad831133741859b5d',
    'name': 'EASY Study - 75 Image, full featureset',
    'description': '',
    'annotations': [ ... list with 602 items],
    'features': [ ... list with 27 items],
    'images': [ ... list with 86 items],
    'questions': [ ... list with 1 items],
    'users': [ ... list with 7 items],})

In [4]:
# print list of study images
study.image_names()

['ISIC_0016090',
 'ISIC_0016091',
 'ISIC_0016092',
 'ISIC_0016093',
 'ISIC_0016094',
 'ISIC_0016095',
 'ISIC_0016096',
 'ISIC_0016097',
 'ISIC_0016098',
 'ISIC_0016099',
 'ISIC_0016100',
 'ISIC_0016101',
 'ISIC_0016102',
 'ISIC_0016103',
 'ISIC_0016104',
 'ISIC_0016105',
 'ISIC_0016106',
 'ISIC_0016107',
 'ISIC_0016108',
 'ISIC_0016109',
 'ISIC_0016110',
 'ISIC_0016111',
 'ISIC_0016112',
 'ISIC_0016113',
 'ISIC_0016114',
 'ISIC_0016115',
 'ISIC_0016116',
 'ISIC_0016117',
 'ISIC_0016118',
 'ISIC_0016119',
 'ISIC_0016120',
 'ISIC_0016121',
 'ISIC_0016122',
 'ISIC_0016123',
 'ISIC_0016124',
 'ISIC_0016125',
 'ISIC_0016126',
 'ISIC_0016127',
 'ISIC_0016128',
 'ISIC_0016129',
 'ISIC_0016130',
 'ISIC_0016131',
 'ISIC_0016132',
 'ISIC_0016133',
 'ISIC_0016134',
 'ISIC_0016135',
 'ISIC_0016136',
 'ISIC_0016137',
 'ISIC_0016138',
 'ISIC_0016139',
 'ISIC_0016140',
 'ISIC_0016141',
 'ISIC_0016142',
 'ISIC_0016143',
 'ISIC_0016144',
 'ISIC_0016145',
 'ISIC_0016146',
 'ISIC_0016147',
 'ISIC_0016148

In [5]:
# Retrieve an annotation
annotation = api.annotation(study.annotations[0])

# print information about the annotation
annotation

isicarchive.annotation.Annotation({
    'id': '58d9189ad831133741859b5f',
    'study_id': '58d9189ad831133741859b5d',
    'study_name': 'EASY Study - 75 Image, full featureset',
    'image_id': '58b0a360d831137d0a388344',
    'image_name': 'ISIC_0016090',
    'user_id': '578e64b09fc3c10d6fd12e4f',
    'user_name': 'User 6YB2',
    'user_lastname': None,
    'features': '{'Globules / Clods : Regular', 'Structureless : Blotch regular'}',})

In [6]:
# Retrieve an image for this study
image = api.image(study.images[0])

# Load the image data (from cache or online)
image.load_imagedata()

# And load the superpixels, and parse into coordinates map
image.load_superpixels()
image.map_superpixels()

# print information
image

isicarchive.image.Image({
    'id': '58b0a360d831137d0a388344',
    'name': 'ISIC_0016090',
    'dataset_id': '5a2ecc5d1165975c94594294',
    'dataset_name': 'EASY: MSK',
    'meta_acquisition': '{'image_type': 'dermoscopic', 'pixelsX': 2592, 'pixelsY': 1936}',
    'meta_clinical': '{'age_approx', 'anatom_site_general', 'benign_malignant', 'diagnosis', 'diagnosis_confirm_type', 'melanocytic', 'nevus_type', 'sex'}',
    'meta_clinical_benign_malignant': 'benign',
    'meta_clinical_diagnosis': 'nevus',
    'meta_clinical_diagnosis_confirm_type': 'single image expert consensus',
    'meta_clinical_melanocytic': True,
    'notes_reviewed': '{'accepted': True, 'time': '2017-02-24T21:34:40.748000+00:00', 'userId': '55d4cde89fc3c1490e995086'}',
    'superpixels_max': 993,
    'superpixels_shape': (1936, 2592),})

In [7]:
# and get some information on the dataset of the image
dataset = api.dataset(image.dataset)
dataset.load_images()
dataset

isicarchive.dataset.Dataset({
    'id': '5a2ecc5d1165975c94594294',
    'name': 'EASY: MSK',
    'description': 'Full image set for MSK EASY Study.',
    'access_level': 2,
    'license': 'CC-0',
    'owner': 'Memorial Sloan Kettering Cancer Center',
    'updated': '2017-04-13T20:09:01.391000+00:00',
    'images': [ ... list with 103 items],
    'images_for_review': [ ... list with 0 items],})